In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geoip2.database
import subprocess
dataset_path_of_city = "/root/jupter_listen/GeoLite2-City_20200901/GeoLite2-City.mmdb"

In [2]:
def getLoc(ip,dataset_path_of_city=dataset_path_of_city):
    reader = geoip2.database.Reader(dataset_path_of_city)
    data = reader.city(ip)
    dict_ = {"ip地址":ip,"国家":data.country.names['en'],"省份":data.subdivisions.most_specific.names['en'],"城市":data.city.names['en'],"纬度":data.location.latitude,"经度":data.location.longitude}
    return dict_

In [3]:
def getNetWork():
    (status, output) = subprocess.getstatusoutput('netstat -atp')
    print(status)
    return_cmd = [line.split() for line in output.splitlines()]
    data = return_cmd[1:]
    columns=["Proto","Recv-Q","Send-Q","Local","Address_IP","Foreign","Address_Program","State","PID/Program","name"]
    df = pd.DataFrame(data,columns=columns)
    df.drop([0],axis=0)
    return df.drop(0,axis=0)
getNetWork()

0


,Proto,Recv-Q,Send-Q,Local,Address_IP,Foreign,Address_Program,State,PID/Program,name
1,tcp,0,0,localhost:59477,*:*,LISTEN,2051/python,None,None,None
2,tcp,0,0,localhost:8888,*:*,LISTEN,1977/python,None,None,None
3,tcp,0,0,localhost:8889,*:*,LISTEN,2462/python,None,None,None
4,tcp,0,0,localhost:59898,*:*,LISTEN,2051/python,None,None,None
5,tcp,0,0,localhost:55419,*:*,LISTEN,20860/python,None,None,None
...,...,...,...,...,...,...,...,...,...,...
87,tcp,0,0,localhost:55205,localhost:38060,ESTABLISHED,2048/python,None,None,None
88,tcp,0,0,localhost:58126,localhost:43060,ESTABLISHED,1977/python,None,None,None
89,tcp,0,0,localhost:46992,localhost:8888,ESTABLISHED,20605/0,None,None,None
90,tcp6,0,0,localhost:8888,[::]:*,LISTEN,1977/python,None,None,None


In [4]:
def getIP():
    IP = getNetWork().Address_IP
    init_dict = {"ip地址":[],"国家":[],"省份":[],"城市":[],"纬度":[],"经度":[]}
    for ip in IP:
        if ip[:3] not in ["0.0","127",":::","loc",'[::','*:*']:
            try:
                IP_dict = getLoc(ip.split(':')[0])
            except Exception as e:
                IP_dict = {'ip地址': ip,'国家': '?','省份': '?','城市': '?','纬度': '?','经度':"?"}
            finally:
                [init_dict[name].append(IP_dict[name]) for name in init_dict]
    return pd.DataFrame(init_dict).drop_duplicates()
getIP()

0


,ip地址,国家,省份,城市,纬度,经度
0,39.107.228.153:https,?,?,?,?,?
1,100.100.30.25:http,?,?,?,?,?
2,183.167.144.148:29942,?,?,?,?,?
3,183.167.144.148:29878,?,?,?,?,?
